# nvImageCodec with cuCIM (Linux only)

In [ ]:
import os
from matplotlib import pyplot as plt
import cupy as cp

Setting resource folder

In [ ]:
resources_dir = os.getenv("PYNVIMGCODEC_EXAMPLES_RESOURCES_DIR", "../assets/images/")

Import nvImageCodec module and create both Decoder and Encoder

In [ ]:
from nvidia import nvimgcodec
decoder = nvimgcodec.Decoder()
encoder = nvimgcodec.Encoder()

Read and decode Jpeg2000 file with nvImageCodec

In [ ]:
nv_img = decoder.read(resources_dir + "cat-1046544_640.jp2")

Convert to cupy ndarray

In [ ]:
cp_img = cp.asarray(nv_img)

In [ ]:
cp_img_single_channel = cp_img[:, :, 1]

In [ ]:
from cucim.skimage.filters import gaussian
cp_img_blurred = gaussian(cp_img_single_channel, sigma=5)
image = cp.asnumpy(cp_img_blurred)
plt.imshow(image, cmap='gray')

In [ ]:
from cucim.skimage.filters import threshold_otsu
# determine threshold
threshold = threshold_otsu(cp_img_blurred)

# binarize image by apply the threshold
cp_binary_gpu = cp_img_blurred > threshold

image = cp.asnumpy(cp_binary_gpu)
plt.imshow(image, cmap='gray')

In [ ]:
from cucim.skimage.morphology import binary_erosion, disk

eroded_gpu = binary_erosion(cp_binary_gpu, selem=disk(2))
image = cp.asnumpy(eroded_gpu)
plt.imshow(image, cmap='gray')


We can pass directly nvImageCodec Image object to cuCIM 

In [ ]:
cp_img_blurred_color = gaussian(nv_img, sigma=5)
image = cp.asnumpy(cp_img_blurred_color)
plt.imshow(image)

Please notice that cuCIM returned blurred color image as a HWC and in float so it must be converted

In [ ]:
print(cp_img_blurred_color.__cuda_array_interface__)
cp_img_blurred_color = cp_img_blurred_color*255
cp_img_blurred_color_int = cp_img_blurred_color.astype("u1")
print(cp_img_blurred_color_int.__cuda_array_interface__)

Pass to nvImageCodec

In [ ]:
nv_img = nvimgcodec.as_image(cp_img_blurred_color_int)

In [ ]:
encoder.write("blurred.jpg", nv_img)

Read back with OpenCV to verify

In [ ]:
import cv2
image = cv2.imread("blurred.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)
